# Drug–Protein Interaction Prediction

**Cross-validation strategies for paired input (protein–drug) data using a KNN model.**

Three cross-validation schemes are implemented and compared:
- **LODO-CV**: Leave-One-Drug-Out
- **LOPO-CV**: Leave-One-Protein-Out
- **LOOPD-CV**: Leave-One-Out-Protein-and-Drug (strictest / most realistic)

## 1. Imports

In [6]:
import numpy as np
import pandas as pd
from scipy import stats
from collections import defaultdict
from statsmodels.stats.multitest import multipletests

from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import LeaveOneOut
from sklearn.metrics import pairwise_distances

import matplotlib.pyplot as plt


## 2. Data Loading

In [7]:
# Load input features, output labels, and protein-drug pair metadata
input_data  = pd.read_csv('input.data',  sep=r'\s+', header=None)
output_data = pd.read_csv('output.data', header=None)
pairs_data  = pd.read_csv('pairs.data',  sep=r'\s+', header=None,
                          quotechar='"', engine='python')

pairs_data.columns = ['protein', 'drug']

print(pairs_data.head(3))
print(f"\nTotal samples : {len(pairs_data)}")
print(f"Unique proteins: {pairs_data['protein'].nunique()}")
print(f"Unique drugs   : {pairs_data['drug'].nunique()}")

# Extract arrays used throughout
proteins = pairs_data['protein'].values
drugs    = pairs_data['drug'].values
y_all    = output_data.values.flatten()
X_all    = input_data.values


  protein   drug
0   "D40"   "T2"
1   "D31"  "T64"
2    "D6"  "T58"

Total samples : 400
Unique proteins: 59
Unique drugs   : 77


## 3. Leave-One-Drug-Out Cross-Validation (LODO-CV)

For each drug, train on all other drugs and evaluate on the held-out drug.
The C-index measures how well the model ranks drug–protein affinities.

In [8]:
# Build drug → row-index lookup
drug_to_indices = defaultdict(list)
for i, drug in enumerate(drugs):
    drug_to_indices[drug].append(i)

unique_drugs = list(drug_to_indices.keys())
print(f"Total unique drugs: {len(unique_drugs)}")

c_indices  = []
y_true_all = []
y_pred_all = []

for test_drug in unique_drugs:
    test_indices  = drug_to_indices[test_drug]
    train_indices = [i for i in range(len(drugs)) if drugs[i] != test_drug]

    X_train = X_all[train_indices]
    y_train = y_all[train_indices]
    X_test  = X_all[test_indices]
    y_test  = y_all[test_indices]

    model = KNeighborsRegressor(n_neighbors=10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)

    # C-index for this drug
    if len(y_pred) > 1:
        y_pred_pairs = pairwise_distances(y_pred.reshape(-1, 1))
        y_test_pairs = pairwise_distances(y_test.reshape(-1, 1))
        concordant = np.sum((y_pred_pairs > 0) & (y_test_pairs > 0))
        discordant = np.sum((y_pred_pairs > 0) & (y_test_pairs < 0))
        ties       = np.sum(y_test_pairs == 0)
        c_index    = (concordant + 0.5 * ties) / (concordant + discordant + ties)
    else:
        c_index = 0.5

    c_indices.append(c_index)
    print(f"Drug {test_drug}: C-index = {c_index:.3f}")

# Overall summary
mean_c  = np.mean(c_indices)
std_c   = np.std(c_indices)
print(f"\nOverall Mean C-index from LODO-CV : {mean_c:.3f}")
print(f"Standard deviation of C-index      : {std_c:.3f}")

# Performance breakdown
perfect = sum(1 for c in c_indices if c == 1.0)
good    = sum(1 for c in c_indices if 0.7 <= c < 1.0)
fair    = sum(1 for c in c_indices if 0.5 <= c < 0.7)
poor    = sum(1 for c in c_indices if c < 0.5)
print(f"\nPerfect (C=1.0): {perfect} drugs")
print(f"Good  (0.7–1.0): {good} drugs")
print(f"Fair  (0.5–0.7): {fair} drugs")
print(f"Poor  (<0.5)   : {poor} drugs")
poor_drugs = [d for d, c in zip(unique_drugs, c_indices) if c < 0.5]
print(f"Drugs with poor prediction: {poor_drugs}")


Total unique drugs: 77
Drug "T2": C-index = 0.875
Drug "T64": C-index = 0.917
Drug "T58": C-index = 0.875
Drug "T49": C-index = 0.833
Drug "T28": C-index = 0.938
Drug "T33": C-index = 0.900
Drug "T60": C-index = 0.833
Drug "T36": C-index = 0.944
Drug "T23": C-index = 0.929
Drug "T43": C-index = 0.929
Drug "T52": C-index = 0.917
Drug "T69": C-index = 0.929
Drug "T66": C-index = 0.900
Drug "T39": C-index = 0.929
Drug "T47": C-index = 0.938
Drug "T50": C-index = 0.917
Drug "T27": C-index = 0.944
Drug "T1": C-index = 0.929
Drug "T26": C-index = 0.750
Drug "T70": C-index = 0.929
Drug "T57": C-index = 0.900
Drug "T32": C-index = 0.750
Drug "T42": C-index = 0.917
Drug "T9": C-index = 0.833
Drug "T63": C-index = 0.950
Drug "T51": C-index = 0.833
Drug "T59": C-index = 0.929
Drug "T72": C-index = 0.944
Drug "T44": C-index = 0.750
Drug "T41": C-index = 0.950
Drug "T75": C-index = 0.833
Drug "T76": C-index = 0.750
Drug "T65": C-index = 0.750
Drug "T45": C-index = 0.958
Drug "T54": C-index = 0.917


## 4. Leave-One-Protein-Out Cross-Validation (LOPO-CV)

For each protein, train on all other proteins and evaluate on the held-out protein.

In [9]:
# Build protein → row-index lookup
protein_to_indices = defaultdict(list)
for i, protein in enumerate(proteins):
    protein_to_indices[protein].append(i)

unique_proteins = list(protein_to_indices.keys())
print(f"Total unique proteins: {len(unique_proteins)}")

c_indices  = []
y_true_all = []
y_pred_all = []

for test_protein in unique_proteins:
    test_indices  = protein_to_indices[test_protein]
    train_indices = [i for i in range(len(proteins)) if proteins[i] != test_protein]

    X_train = X_all[train_indices]
    y_train = y_all[train_indices]
    X_test  = X_all[test_indices]
    y_test  = y_all[test_indices]

    model = KNeighborsRegressor(n_neighbors=10)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    y_true_all.extend(y_test)
    y_pred_all.extend(y_pred)

    # C-index for this protein
    if len(y_pred) > 1:
        y_pred_pairs = pairwise_distances(y_pred.reshape(-1, 1))
        y_test_pairs = pairwise_distances(y_test.reshape(-1, 1))
        concordant = np.sum((y_pred_pairs > 0) & (y_test_pairs > 0))
        discordant = np.sum((y_pred_pairs > 0) & (y_test_pairs < 0))
        ties       = np.sum(y_test_pairs == 0)
        c_index    = (concordant + 0.5 * ties) / (concordant + discordant + ties)
    else:
        c_index = 0.5

    c_indices.append(c_index)
    print(f"Protein {test_protein}: C-index = {c_index:.3f}")

# Overall summary
mean_c = np.mean(c_indices)
std_c  = np.std(c_indices)
print(f"\nOverall Mean C-index from LOPO-CV : {mean_c:.3f}")
print(f"Standard deviation of C-index      : {std_c:.3f}")


Total unique proteins: 59
Protein "D40": C-index = 0.944
Protein "D31": C-index = 0.935
Protein "D6": C-index = 0.926
Protein "D56": C-index = 0.875
Protein "D20": C-index = 0.926
Protein "D46": C-index = 0.922
Protein "D50": C-index = 0.952
Protein "D37": C-index = 0.857
Protein "D34": C-index = 0.500
Protein "D3": C-index = 0.938
Protein "D45": C-index = 0.946
Protein "D59": C-index = 0.891
Protein "D1": C-index = 0.926
Protein "D39": C-index = 0.900
Protein "D22": C-index = 0.935
Protein "D58": C-index = 0.906
Protein "D8": C-index = 0.875
Protein "D44": C-index = 0.878
Protein "D10": C-index = 0.857
Protein "D27": C-index = 0.926
Protein "D38": C-index = 0.931
Protein "D11": C-index = 0.905
Protein "D2": C-index = 0.952
Protein "D30": C-index = 0.936
Protein "D5": C-index = 0.891
Protein "D14": C-index = 0.957
Protein "D55": C-index = 0.931
Protein "D54": C-index = 0.900
Protein "D17": C-index = 0.951
Protein "D9": C-index = 0.919
Protein "D4": C-index = 0.893
Protein "D19": C-inde

## 5. Leave-One-Out-Protein-and-Drug Cross-Validation (LOOPD-CV)

The strictest scheme: for each unique (protein, drug) pair in the dataset,
train on rows where **neither** that protein **nor** that drug appears.
This simulates predicting interactions for completely unseen proteins and drugs.

In [10]:
X_real = input_data.values
y_real = output_data.values.flatten()
model  = KNeighborsRegressor(n_neighbors=10)

y_true_all = []
y_pred_all = []

for test_protein in unique_proteins:
    for test_drug in unique_drugs:
        # Test set: only the specific (protein, drug) pair
        test_indices = [i for i in range(len(proteins))
                        if proteins[i] == test_protein and drugs[i] == test_drug]
        if len(test_indices) == 0:
            continue  # pair not in dataset

        # Training set: exclude ALL rows that involve the test protein OR test drug
        train_indices = [i for i in range(len(proteins))
                         if proteins[i] != test_protein and drugs[i] != test_drug]
        if len(train_indices) == 0:
            continue

        X_train = X_real[train_indices]
        y_train = y_real[train_indices]
        X_test  = X_real[test_indices]
        y_test  = y_real[test_indices]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        y_true_all.extend(y_test)
        y_pred_all.extend(y_pred)

# Overall C-index
y_true_arr = np.array(y_true_all)
y_pred_arr = np.array(y_pred_all)

concordant = discordant = ties = 0
for i in range(len(y_true_arr)):
    for j in range(i + 1, len(y_true_arr)):
        if y_true_arr[i] == y_true_arr[j]:
            continue
        if y_pred_arr[i] > y_pred_arr[j] and y_true_arr[i] > y_true_arr[j]:
            concordant += 1
        elif y_pred_arr[i] < y_pred_arr[j] and y_true_arr[i] < y_true_arr[j]:
            concordant += 1
        elif y_pred_arr[i] == y_pred_arr[j]:
            ties += 1
        else:
            discordant += 1

overall_c_index = (concordant + 0.5 * ties) / (concordant + discordant + ties)
print(f"Overall C-index from LOOPD-CV: {overall_c_index:.3f}")


Overall C-index from LOOPD-CV: 0.522


## 6. Results Summary

| Cross-Validation Scheme | Mean C-index | Std Dev |
|-------------------------|:------------:|:-------:|
| LODO-CV                 | 0.874        | 0.083   |
| LOPO-CV                 | 0.894        | 0.082   |
| LOOPD-CV                | 0.522        | —       |

The naïve LOO reported in the scientist's letter (problem statement) gave >90%, while the proper LOOPD gives only ~52%.
